In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

from astroquery.gaia import Gaia
Gaia.ROW_LIMIT = 10000 # Set the row limit for returned data 

Created TAP+ (v20200428.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg)
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

In [3]:
SkyCoord('00h48m26.4s', '85d15m36s', frame = 'icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

In [4]:
ngc188_center.ra, ngc188_center.dec

(<Longitude 12.11 deg>, <Latitude 85.26 deg>)

In [5]:
ngc188_center.ra

<Longitude 12.11 deg>

In [6]:
ngc188_center.dec

<Latitude 85.26 deg>

In [7]:
type(ngc188_center.ra), type(ngc188_center.dec)

(astropy.coordinates.angles.Longitude, astropy.coordinates.angles.Latitude)

In [8]:
(ngc188_center.ra.to(u.hourangle),
ngc188_center.ra.to(u.radian),
ngc188_center.ra.to(u.degree))

(<Longitude 0.80733333 hourangle>,
 <Longitude 0.21135937 rad>,
 <Longitude 12.11 deg>)

In [9]:
(ngc188_center.ra.hour, ngc188_center.ra.radian,
ngc188_center.ra.degree)

(0.8073333333333335, 0.21135937241651329, 12.11)

In [10]:
ngc188_center.ra.to_string(unit = u.hourangle, sep = ':', pad = True)

'00:48:26.4'

In [11]:
job = Gaia.cone_search_async(ngc188_center, radius = 0.5*u.deg)
ngc188_table = job.get_results()

#only keep stars brighter than G=19 magnitude
ngc188_table = ngc188_table[ngc188_table['phot_g_mean_mag'] < 19*u.mag]

INFO: Query finished. [astroquery.utils.tap.core]


In [12]:
cols = ['source_id',
       'ra', 
       'dec',
       'parallax',
       'pmra',
       'pmdec',
       'radial_velocity',
       'phot_g_mean_mag',
       'phot_bp_mean_mag',
       'phot_rp_mean_mag']
ngc188_table[cols].write('gaia_results.fits', overwrite = True)

In [13]:
ngc188_table = QTable.read('gaia_results.fits')

In [14]:
len(ngc188_table)

4934

In [15]:
ngc188_table['ra']

<Quantity [12.12843989, 12.09381253, 12.09832966, ..., 16.0553526 ,
           16.05884572, 13.05668027] deg>

In [16]:
ngc188_table['dec']

<Quantity [85.26076889, 85.26522648, 85.25443128, ..., 85.64251242,
           85.6423048 , 84.7728428 ] deg>

In [18]:
ngc188_gaia_coords = SkyCoord(ngc188_table['ra'], ngc188_table['dec'])

ngc188_gaia_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(12.12843989, 85.26076889), (12.09381253, 85.26522648),
     (12.09832966, 85.25443128), ..., (16.0553526 , 85.64251242),
     (16.05884572, 85.6423048 ), (13.05668027, 84.7728428 )]>

In [19]:
ngc188_center = SkyCoord.from_name('NGC 188')

In [20]:
ngc188_center.to_string(style = "hmsdms", sep = ":", precision = 1)

'00:48:26.0 +85:15:18.0'

In [22]:
ngc188_gaia_coords.separation(ngc188_center)

<Angle [0.00600359, 0.01029663, 0.00100414, ..., 0.49803955, 0.49805666,
        0.4891353 ] deg>

In [23]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg, distance = 1.96*u.kpc)

In [25]:
parallax_snr = ngc188_table['parallax'] / ngc188_table['parallax_error']
ngc188_table_3d = ngc188_table[parallax_snr > 10]
len(ngc188_table_3d)

KeyError: 'parallax_error'